In [1]:
# import the optimize solver Gurobi
import gurobipy as gp
from gurobipy import * 

In [2]:
m = Model() # Import and create the model

Using license file C:\Users\chaoz\gurobi_licence\gurobi.lic
Academic license - for non-commercial use only


In [3]:
# Set the input Parameter: 
avg_speed = 5  # The average speed at which he rides is 5 ft/sec 
stop_time = 3  # Stop sign takes 3 sec.
traffic_light_time = 45 # Traffic red light takes 45 sec.

In [4]:
# Set each node
nodes = ['12Fifty5', 'Orange St Jn', 'University TC', 'Lemon St L', 'Lemon St Jn', 'HH Junction', 'DB Junction', 'ASU CP', 'SDFC Side', 'MU Side', 'MCRD']

# Set required net outflow

net_flow = {}
for i in range(len(nodes)):
    if i == 0:
        net_flow[nodes[i]] = 1
    elif i == len(nodes) - 1:
        net_flow[nodes[i]] = -1
    else:
        net_flow[nodes[i]] = 0


In [5]:
# Set arcs, distance from orign to destination, stop sign and traffic light

arcs, distance, stop_sign, traffic_light = gp.multidict({
    ('12Fifty5', 'Orange St Jn'):   [300, 1, 0 ],
    ('12Fifty5', 'University TC'):  [4000, 1, 1],
    ('Orange St Jn', 'Lemon St L'): [465, 1, 0],
    ('Orange St Jn',  'Lemon St Jn'):  [500, 0, 0],
    ('University TC',  'DB Junction'): [1500, 0, 1],
    ('Lemon St L',  'Lemon St Jn'):  [10, 0, 0],
    ('Lemon St Jn', 'HH Junction'):   [100, 0, 1],
    ('HH Junction', 'DB Junction'):  [500, 0, 1],
    ('HH Junction', 'University TC'):  [1700, 0, 1],
    ('DB Junction',  'ASU CP'):   [300, 1, 0],
    ('ASU CP',  'SDFC Side'): [500, 1, 0],
    ('ASU CP',  'MU Side'):  [700, 0, 0],
    ('SDFC Side',  'MCRD'):   [0, 0, 0],
    ('MU Side',  'MCRD'): [0, 0, 0]})

In [6]:
# Caculate travel time for each arcs
travel_time = {}
for key in arcs:
    travel_time[key] = distance[key]/avg_speed + stop_sign[key] * stop_time + traffic_light_time* traffic_light[key]

In [7]:
m.update()

In [8]:
# Create variables
flow = m.addVars(arcs, obj = travel_time, name="flow")

In [12]:
# Flow-conservation constraints
m.addConstrs(
    (flow.sum(i, '*') - flow.sum('*', i) == net_flow[i]
        for i in nodes), "node")

{'12Fifty5': <gurobi.Constr *Awaiting Model Update*>,
 'Orange St Jn': <gurobi.Constr *Awaiting Model Update*>,
 'University TC': <gurobi.Constr *Awaiting Model Update*>,
 'Lemon St L': <gurobi.Constr *Awaiting Model Update*>,
 'Lemon St Jn': <gurobi.Constr *Awaiting Model Update*>,
 'HH Junction': <gurobi.Constr *Awaiting Model Update*>,
 'DB Junction': <gurobi.Constr *Awaiting Model Update*>,
 'ASU CP': <gurobi.Constr *Awaiting Model Update*>,
 'SDFC Side': <gurobi.Constr *Awaiting Model Update*>,
 'MU Side': <gurobi.Constr *Awaiting Model Update*>,
 'MCRD': <gurobi.Constr *Awaiting Model Update*>}

In [13]:
# Compute optimal solution
m.optimize()

Gurobi Optimizer version 9.0.1 build v9.0.1rc0 (win64)
Optimize a model with 11 rows, 14 columns and 28 nonzeros
Model fingerprint: 0x45a2a77f
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+00, 8e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 8 rows and 9 columns
Presolve time: 0.02s
Presolved: 3 rows, 5 columns, 10 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    3.9200000e+02   2.000000e+00   0.000000e+00      0s
       1    5.3700000e+02   0.000000e+00   0.000000e+00      0s

Solved in 1 iterations and 0.48 seconds
Optimal objective  5.370000000e+02


In [14]:
# m.printAttr('X')


    Variable            X 
-------------------------
flow[12Fifty5,Orange St Jn]            1 
flow[Orange St Jn,Lemon St L]            1 
flow[Lemon St L,Lemon St Jn]            1 
flow[Lemon St Jn,HH Junction]            1 
flow[HH Junction,DB Junction]            1 
flow[DB Junction,ASU CP]            1 
flow[ASU CP,SDFC Side]            1 
flow[SDFC Side,MCRD]            1 


In [19]:
# Print solution
if m.status == GRB.OPTIMAL:
    solution = m.getAttr('x', flow)
    for i, j in arcs:
        if solution[i,j] > 0:
            print('%s -> %s : %g' % (i, j, travel_time[i,j]))

12Fifty5 -> Orange St Jn : 63
Orange St Jn -> Lemon St L : 96
Lemon St L -> Lemon St Jn : 2
Lemon St Jn -> HH Junction : 65
HH Junction -> DB Junction : 145
DB Junction -> ASU CP : 63
ASU CP -> SDFC Side : 103
SDFC Side -> MCRD : 0


In [20]:
# Get the Optimal Objective Value
m.ObjVal

537.0